![](../images/rivacon_frontmark_combined_header.png)

# Deposits and FRAs 

In [ ]:
import pyvacon
import datetime as dt
from rivapy.marketdata.curves import DiscountCurve
from rivapy.instruments import DepositSpecification, ForwardRateAgreementSpecification
from rivapy.pricing import price
from pyvacon.finance.pricing import DepositPricer, ForwardRateAgreementPricer, ForwardRateAgreementPricingData, PricingRequest
import math
%matplotlib inline

## Definition of a deposit

A deposit is simply a short-term zero-coupon bond where at the start date the lender pays the nominal amount $N$ to the borrower, and at maturity the borrower pays back the nominal amount plus the accrued interest. Interest is usually expressed using simple compounding, i.e. the amount paid back at maturity is $N \cdot (1 + r \cdot \tau)$ where $r$ is the interest rate and $\tau$ is the year fraction between start and end dates.


In [ ]:
# Set up deposit specification

ccy = 'EUR'
ref_date = dt.datetime(2023, 10, 26)
start_date = ref_date
end_date = dt.datetime(2024, 1, 26)
dcc = 'Act360'
rate = .03
notional = 100.
deposit_spec = DepositSpecification('dummy_id', 'dummy_issuer', 'NONE', ccy, ref_date, start_date, end_date, notional, rate, dcc)


In [ ]:
# Set up (flat) curve
object_id = "TEST_CURVE"
flat_rate = 0.025

days_to_maturity = [1, 180, 365, 720, 3 * 365, 4 * 365, 10 * 365]
dates = [ref_date + dt.timedelta(days=d) for d in days_to_maturity]

df = [math.exp(-d / 365.0 * flat_rate) for d in days_to_maturity]
dc = DiscountCurve(id=object_id, refdate=ref_date, dates=dates, df=df)

In [ ]:
# Determine fair rate
fair_rate = DepositPricer.impliedSimplyCompoundedRate(
    ref_date, dc._get_pyvacon_obj(), None, deposit_spec)
print(fair_rate)

In [ ]:
# Check fair rate
# discount factor from curve
df_curve = dc.value(ref_date, end_date)
# discount factor implied by fair rate (simple compounding, Act/360)
df_implied = 1. / (1. + (end_date - start_date).total_seconds() / (360. * 86400) * fair_rate)
print(df_curve)
print(df_implied)

## Definition of a forward rate agreement (FRA)

A forward rate agreement is in principle a forward-starting deposit for a time period $[T_1, T_2]$ in the future (which is concluded at a start date $T_0$). However, rather than actually entering into a deposit at time $T_1$, an FRA is "cash-settled" at $T_1$ by exchanging the present value of the agreement (using the fixing of the underlying index to discount the cash flows from $T_2$ to $T_1$):
$$PV(T_1) = N \tau \frac {r - K} {1 + r\cdot \tau}$$
where
- $N$ is the nominal amount
- $\tau$ is the year fraction between $T_1$ and $T_2$
- $r$ is the index fixing at $T_1$ for the period $[T_1, T_2]$
- $K$ is the forward rate agreed upon at $T_0$

In [ ]:
# Set up FRA
ccy = 'EUR'
fra_rate = .04
ref_date = dt.datetime(2023, 10, 26)
start_date = dt.datetime(2024, 1, 26)
end_date = dt.datetime(2024, 4, 26)

fra = ForwardRateAgreementSpecification(
    'dummy_id', 'dummy_issuer', 'NONE', ccy, notional,end_date, fra_rate, 'dummy_udl', True,
    start_date, end_date, start_date, end_date, 'Act360', 'Act360', ''
)


In [ ]:
# Set up curves for pricing
# Discount curve
object_id = "TEST_DC"
dsc_rate = 0.01
df = [math.exp(-d / 365.0 * dsc_rate) for d in days_to_maturity]
dc = DiscountCurve(id=object_id, refdate=ref_date, dates=dates, df=df)

# Fixing curve
object_id = "TEST_fwd"
fwd_rate = 0.05
fwd_df = [math.exp(-d / 365.0 * fwd_rate) for d in days_to_maturity]
fwd_dc = DiscountCurve(id=object_id, refdate=ref_date, dates=dates, df=fwd_df)

In [ ]:
# Price FRA
fra_pricing_data = ForwardRateAgreementPricingData()
fra_pricing_data.pricer = 'ForwardRateAgreementPricer'
fra_pricing_data.valDate = ref_date
fra_pricing_data.pricingRequest = PricingRequest()
fra_pricing_data.discountCurve = dc._get_pyvacon_obj()
fra_pricing_data.forwardCurve = fwd_dc._get_pyvacon_obj()
fra_pricing_data.spec = fra

fra_price = price(fra_pricing_data).getPrice()
print(f'Price: {fra_price}')

In [ ]:
# Calculate fair FRA rate (from froward curve)
fair_rate = ForwardRateAgreementPricer.computeFairRate(ref_date, fra, fwd_dc._get_pyvacon_obj())
print(fair_rate)

In [ ]:
# Check that "fair" FRA has indeed price 0
fair_fra = ForwardRateAgreementSpecification(
    'dummy_id', 'dummy_issuer', 'NONE', ccy, notional,end_date, fair_rate, 'dummy_udl', True,
    start_date, end_date, start_date, end_date, 'Act360', 'Act360', ''
)
fra_pricing_data = ForwardRateAgreementPricingData()
fra_pricing_data.pricer = 'ForwardRateAgreementPricer'
fra_pricing_data.valDate = ref_date
fra_pricing_data.pricingRequest = PricingRequest()
fra_pricing_data.discountCurve = dc._get_pyvacon_obj()
fra_pricing_data.forwardCurve = fwd_dc._get_pyvacon_obj()
fra_pricing_data.spec = fair_fra

fra_price = price(fra_pricing_data).getPrice()
print(f'Price: {fra_price}')